In [1]:
using IPUToolkit.IPUCompiler
using IPUToolkit.Poplar

# Get a device, and initialise the graph and the program
device = Poplar.get_ipu_device()
target = Poplar.DeviceGetTarget(device)
graph = Poplar.Graph(target)
prog = Poplar.ProgramSequence()

# Multiply input vector `in` by 2, and store result in vector `out`.
@codelet graph function TimesTwo(in::VertexVector{Float16, IPUCompiler.In}, out::VertexVector{Float16, IPUCompiler.Out})
    out .= in .* 2
end

# Copy the input array `in` into `out` and sort the latter in-place.
# Also, print out how many cycles it took to sort the vector.
@codelet graph function Sort(in::VertexVector{Float16, IPUCompiler.In}, out::VertexVector{Float16, IPUCompiler.Out})
    copyto!(out, in)
    @ipushowcycles sort!(out)
end

# Create the input vector on the IPU
inconst = Poplar.GraphAddConstant(graph, Float16[5, 2, 10, 102, -10, 2, 256, 15, 32, 100])
# Create other variable vectors on the IPU
outvec1 = similar(graph, inconst, "outvec1");
outvec2 = similar(graph, inconst, "outvec2");

# Add vertices to the graph, running on a single tile
add_vertex(graph, prog, TimesTwo, inconst, outvec1)
add_vertex(graph, prog, Sort, outvec1, outvec2)

# Print out the tensors
Poplar.ProgramSequenceAdd(prog, Poplar.ProgramPrintTensor("Input", inconst))
Poplar.ProgramSequenceAdd(prog, Poplar.ProgramPrintTensor("Output Times2", outvec1))
Poplar.ProgramSequenceAdd(prog, Poplar.ProgramPrintTensor("Output Sorted", outvec2))

# Run the program
engine = Poplar.Engine(graph, prog)
Poplar.EngineLoadAndRun(engine, device)

# Detach the device
Poplar.detach_devices()

[ Info: Trying to attach to device 0...
[ Info: Successfully attached to device 0
✓ Compiling codelet TimesTwo: 	 Time: 0:00:04
✓ Compiling codelet Sort: 	 Time: 0:00:01
T[0.3]: sort!(out):
T[0.3]: 9630 cycles
Input: {5,2,10,102,-10,2,256,15,32,100}
Output Times2: {10,4,20,204,-20,4,512,30,64,200}
Output Sorted: {-20,4,4,10,20,30,64,200,204,512}


In [2]:
# Always remember to release the device after use!
Poplar.detach_devices()